In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from datetime import timedelta

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
pd.set_option('display.max_columns', None)
import gc
import warnings
warnings.filterwarnings('ignore')


In [3]:
data = pd.concat([pd.read_parquet('/campaign_data/Dheeraj/Revisitor_Data/For_HP/Data/vistor_info_data_November_2023.parquet.gzip'),pd.read_parquet('/campaign_data/Dheeraj/Revisitor_Data/For_HP/Data/vistor_info_data_October_2023.parquet.gzip'),pd.read_parquet('/campaign_data/Dheeraj/Revisitor_Data/For_HP/Data/vistor_info_data_September_2023.parquet.gzip'),pd.read_parquet('/campaign_data/Dheeraj/Revisitor_Data/For_HP/Data/vistor_info_data_August_2023.parquet.gzip'),pd.read_parquet('/campaign_data/Dheeraj/Revisitor_Data/For_HP/Data/vistor_info_data_July_2023.parquet.gzip')])

In [5]:
data

,userId,visitType,visitTimeStamp,visitTime
0,00000ebf-7fdb-4d32-8cb2-7558bfa7307e,REVISIT,2023-11-01 08:24:58,2023-11-01
1,00000f4f-1c1a-4b56-99ea-526ef1c14182,REVISIT,2023-11-08 14:46:57,2023-11-08
2,00000fc4-c191-48e9-b615-f00e9db1fc37,REVISIT,2023-11-22 10:58:55,2023-11-22
3,000011f7-fbd6-4e04-9b39-1c53b38941d7,REVISIT,2023-11-05 10:14:51,2023-11-05
4,0000124c-d657-4682-9aaa-f09b0043c5e7,REVISIT,2023-11-02 19:51:51,2023-11-02
...,...,...,...,...
4389479,fffff37b-cdc8-49a4-b5eb-01f79d792023,REVISIT,2023-07-22 12:10:34,2023-07-22
4389480,fffff751-f22d-47cb-a134-27cda40c28ec,REVISIT,2023-07-01 10:31:25,2023-07-01
4389481,fffff984-12b6-4477-8543-4c670743b89a,REVISIT,2023-07-15 07:43:16,2023-07-15
4389482,fffff9f2-d147-4c16-b82c-226ac5f88a1a,REVISIT,2023-07-30 21:40:45,2023-07-30


In [14]:
data.visitTime = pd.to_datetime(data.visitTime, errors = 'coerce')

In [6]:
data = data[data['visitTime'].between('2023-07-01','2023-11-30', inclusive = True)]

In [7]:
data

,userId,visitType,visitTimeStamp,visitTime
0,00000ebf-7fdb-4d32-8cb2-7558bfa7307e,REVISIT,2023-11-01 08:24:58,2023-11-01
1,00000f4f-1c1a-4b56-99ea-526ef1c14182,REVISIT,2023-11-08 14:46:57,2023-11-08
2,00000fc4-c191-48e9-b615-f00e9db1fc37,REVISIT,2023-11-22 10:58:55,2023-11-22
3,000011f7-fbd6-4e04-9b39-1c53b38941d7,REVISIT,2023-11-05 10:14:51,2023-11-05
4,0000124c-d657-4682-9aaa-f09b0043c5e7,REVISIT,2023-11-02 19:51:51,2023-11-02
...,...,...,...,...
4389479,fffff37b-cdc8-49a4-b5eb-01f79d792023,REVISIT,2023-07-22 12:10:34,2023-07-22
4389480,fffff751-f22d-47cb-a134-27cda40c28ec,REVISIT,2023-07-01 10:31:25,2023-07-01
4389481,fffff984-12b6-4477-8543-4c670743b89a,REVISIT,2023-07-15 07:43:16,2023-07-15
4389482,fffff9f2-d147-4c16-b82c-226ac5f88a1a,REVISIT,2023-07-30 21:40:45,2023-07-30


In [8]:


# Assuming 'data' is your DataFrame and 'visitTimeStamp' is of datetime type
data['visitTime'] = pd.to_datetime(data['visitTime'])

# Get the maximum date for reference (you can replace this with a specific date if needed)
max_date = data['visitTime'].max()

# Define the date ranges
date_ranges = {
    "last_30_days_REVISIT": (max_date - timedelta(days=30), max_date),
    "30_60_days_REVISIT": (max_date - timedelta(days=60), max_date - timedelta(days=30)),
    "60_90_days_REVISIT": (max_date - timedelta(days=110), max_date - timedelta(days=60))
}

# Create the new columns for each date range
for column, (start_date, end_date) in date_ranges.items():
    data[column] = data['visitTime'].between(start_date, end_date)

# Group by 'userid' and sum each of the new columns
result = data.groupby('userId')[list(date_ranges.keys())].sum()


In [9]:
result

,last_30_days_REVISIT,30_60_days_REVISIT,60_110_days_REVISIT
userId,,,
00000309-3bf5-4aac-8f8b-1c866cdc2e8d,0,0,1
000005b6-f5bf-4dca-83f5-e0ccfd2d83da,0,1,1
000005c2-9810-4a3b-9ac4-4ea4847f2c46,0,1,0
000005c4-b320-4fc5-aa71-56f573148aaa,0,1,0
00000838-0457-567f-bfe1-643274dc0385,0,0,0
...,...,...,...
fffff984-12b6-4477-8543-4c670743b89a,1,0,1
fffff9f2-d147-4c16-b82c-226ac5f88a1a,1,1,1
fffffc03-4e46-5483-9550-11dd19bcfc4b,0,0,0


In [10]:
data = data.groupby(by = 'userId', axis =0).agg({'userId':'count'}).rename(columns={'userId': 'count'})

In [11]:
mapping = pd.concat([pd.read_parquet('/pb_analytics/analytics/240/Varun/BU_Dump/bureau_files/bu_p1.parquet', columns = ['userId', 'mobilePhone']),pd.read_parquet('/pb_analytics/analytics/240/Varun/BU_Dump/bureau_files/bu_p2.parquet', columns = ['userId', 'mobilePhone']),pd.read_parquet('/pb_analytics/analytics/240/Varun/BU_Dump/bureau_files/bu_p3.parquet', columns = ['userId', 'mobilePhone'])])

In [12]:
check = pd.merge(result, mapping, on = 'userId', how = 'inner')

In [13]:
check = check.drop(['userId'], axis = 1)

In [14]:
check

,last_30_days_REVISIT,30_60_days_REVISIT,60_110_days_REVISIT,mobilePhone
0,0,0,1,7600339245
1,0,1,1,8180012955
2,0,1,0,7899535439
3,0,1,0,9769253777
4,0,0,0,9094641505
...,...,...,...,...
10175592,1,0,1,8051234764
10175593,1,1,1,9413402759
10175594,0,0,0,9910182711
10175595,0,1,0,9389553100


In [ ]:
check.to_parquet('revisits_3features.parquet.gzip', compression = 'gzip', index = False)